# General version of data cleaning

This code will automatically perform data cleaning on each of the dataset in the directoty, to run this notebook you need to download all the datasets and place them in the current directoty with the notebook.

Because we automatically find columns and fix them, we can not afford clustering to find outliers for each datasets so remove it in this notebook, the precision and recall rate might be influenced.

Run the code in oreder to perform data clean, cleaned data will be save in the current directoty, and to calculate the precision and recall rate, you will need to manually inspect the result.

In [1]:
import openclean
import glob
import pandas as pd
import numpy as np
import re

In [2]:
import gzip
import humanfriendly
import os

from openclean.data.source.socrata import Socrata

In [3]:
import pandas as pd
from openclean.pipeline import stream

In [4]:
# dataset_list = ['hg8x-zxpr','w9ak-ipjd','dm9a-ab7w','bx6-tga4','ipu4-2q9a','xubg-57si', 'bty7-2jhb', 'hcir-3275, 'pitm-atqc','iz2q-9x8d']
# dataset_names = ['Housing New York Units by Building',
#                  'DOB NOW: Build – Job Application Filings',
#                  'DOB NOW: Electrical Permit Applications', 
#                  'DOB NOW: Build – Approved Permits',
#                  'DOB Permit Issuance', 
#                  'DOB NOW: Safety – Facades Compliance Filings',
#                  'Historical DOB Permit Issuance', 
#                  'Buildings Selected for the Alternative Enforcement Program (AEP)', 
#                  'Open Restaurant Applications', 
#                  'DOB Cellular Antenna Filings']

# dataset = Socrata().dataset('pitm-atqc')
# datafile = './pitm-atqc.tsv.gz'

# if not os.path.isfile(datafile):
#     with gzip.open(datafile, 'wb') as f:
#         print('Downloading ...\n')
#         dataset.write(f)


# fsize = humanfriendly.format_size(os.stat(datafile).st_size)
# print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

In [5]:
# DOB Cellular Antenna Filings: iz2q-9x8d
# Open Restaurant Applications: pitm-atqc
# DOB NOW: Safety – Facades Compliance Filings: xubg-57si
# DOB Permit Issuance: ipu4-2q9a



In [2]:
#file_list = glob.glob("*.tsv.gz")
file_list =['ipu4-2q9a.tsv.gz','iz2q-9x8d.tsv.gz','pitm-atqc.tsv.gz', 'xubg-57si.tsv.gz']

In [3]:
file_list

['ipu4-2q9a.tsv.gz',
 'iz2q-9x8d.tsv.gz',
 'pitm-atqc.tsv.gz',
 'xubg-57si.tsv.gz']

In [8]:
def readData(file):
    
    datafile = './'+file
    
    df  = pd.read_csv(datafile, dtype='object', sep='\t')
    #ds = stream(datafile)
    
    return datafile, df#, ds

In [9]:
def fixColumnNames(df):
    rename_list = list(df.columns)
    rename_dict = dict()

    for i in rename_list:
        col_name = str(i)

        col_name = col_name.strip().replace("_", " ").replace("’", "'").replace(".", "")


        # https://stackoverflow.com/questions/2277352/split-a-string-at-uppercase-letters
        # Split on upper case to seperate cocnatenated words:
        if (not col_name.islower()) and (not col_name.isupper()) and (col_name.find(" ") == -1):
            col_name = " ".join(re.sub("([A-Z])", r" \1", col_name).split())


        if col_name.islower(): 
            col_name = col_name.title()

        if (col_name.isupper()) and (col_name.find(" ") != -1):
            col_name = col_name.title()

        col_name = col_name.replace("No.", "Number")
        col_name = col_name.replace("#", "Number")

        rename_dict[i] = col_name
    return rename_dict    

In [10]:
def findColumns(df, column_name_list):
    data_cols = []

    for col in df.columns:
        for name in column_name_list:
            if name.lower() in col.lower():
                data_cols.append(col)
                
    return  data_cols        

In [11]:
def fix_House_Number_Column(df, col):
    df[col] = df[col].str.replace(pat='(?P<one>\\d)(?P<two>[A-Z]+)', repl='\g<one> \g<two>', regex=True)
    df[col] = df[col].str.replace(pat='(?P<one>GAR$)', repl='GARAGE', regex=True)
    df[col] = df[col].str.replace(pat='NORTH([A-Z]+)?', repl='', regex=True)
    df[col] = df[col].str.replace(pat='EAST([A-Z]+)?', repl='', regex=True)
    df[col] = df[col].str.replace(pat='SOUTH([A-Z]+)?', repl='', regex=True)
    df[col] = df[col].str.replace(pat='WEST([A-Z]+)?', repl='', regex=True)
    df.loc[(~df[col].str.contains('\\d', regex=True)), col] = ''


In [12]:
def fix_Phone_Number_Column(df, col):
    
    df[col] = df[col].str.replace(pat="[^\\d]", repl="", regex=True)    
    df[col] = df[col].str.lstrip('1')
    df.loc[df[col].str.len()!=10, col] = ''

    #return data_cols

In [13]:
def fix_ID_Number_Column(df):
    data_cols = findColumns(df, ['number', 'Number',' No',' NO'])
    
    for col in data_cols:
        
        df[col].fillna('', inplace=True)
        df[col] = df[col].astype('str')
        df[col] = df[col].str.upper()

        df.loc[df[col].str.strip('')=='ONE', col] = '1'
        df.loc[df[col].str.strip('')=='TWO', col] = '2'
        df.loc[df[col].str.strip('')=='THREE', col] = '3'
        df.loc[df[col].str.strip('')=='FOUR', col] = '4'
        df.loc[df[col].str.strip('')=='FIVE', col] = '5'
        df.loc[df[col].str.strip('')=='SIX', col] = '6'
        df.loc[df[col].str.strip('')=='SEVEN', col] = '7'
        df.loc[df[col].str.strip('')=='EIGHT', col] = '8'
        df.loc[df[col].str.strip('')=='NINE', col] = '9'

        df.loc[df[col].str.strip('')=='NONE', col] = ''
        df.loc[df[col].str.strip('')=='none', col] = ''
        df.loc[df[col].str.strip('')=='None', col] = ''

        df.loc[df[col].str.strip('')=='NAN', col] = ''
        df.loc[df[col].str.strip('')=='nan', col] = ''
        df.loc[df[col].str.strip('')=='NaN', col] = ''
        
        df.loc[df[col].str.strip('')=='NO NUMBER', col] = ''
    
        if ('house' in col.lower()) or ('building' in col.lower()):
            fix_House_Number_Column(df, col)
        if ("phone" in col.lower()):
            fix_Phone_Number_Column(df, col)
    
    return data_cols

In [14]:
def fix_Binary_Column(df):
    #data_cols = findColumns(ds, ['Landmarked','Owned', 'Filled'])
    #
    data_cols = []

    for col in df.columns:
        # If column has small # of values, and potentially boolean values like Yes/ No, Y/N, X, True, etc.
        if (df[col].nunique() < 5) and (df[df[col].str.lower()==('y|yes|x|true|n|no|false')][col].count() != 0):
            data_cols.append(col)
        
        
    for col in data_cols:
        
        df.loc[df[col].str.lower().isin(['y','yes', 'x', 'true']), col] = True
        df.loc[df[col].str.lower().isin(['n', 'no', 'nan','false']), col] = False
        df.loc[df[col].str.lower()=='', col] = False
        df[col].fillna(False, inplace=True)
        
        if df[col].nunique()==2:
            df[col] = df[col].astype('bool')
        
    return data_cols

In [15]:
def fix_Monetary_Column(df):
    data_cols = findColumns(df, ['Cost','cost', 'fee', 'Fee'])
    
    for col in data_cols:
        
        df[col] = df[col].astype('str')
        df[col] = df[col].str.replace("$", '', regex=False)
        df[col] = df[col].str.replace("-", '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return data_cols

In [16]:
def fix_Numerical_Column(df):
    data_cols = findColumns(df, ['Units','units', 'Height', 'height', 'Length', 'length', 'Footage', 'footage', 'Sqft', 'sqft'])
    
    for col in data_cols:
        df[col] = df[col].astype('str')

        df[col] = df[col].str.replace('-', '', regex=False)
        df[col] = df[col].str.replace('NONE', '0', regex=False)
        df[col] = df[col].str.replace('none', '0', regex=False)
        df[col] = df[col].str.replace('NAN', '0', regex=False)
        df[col] = df[col].str.replace('NaN', '0', regex=False)
        df[col] = df[col].str.replace('nan', '0', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    return data_cols

In [17]:
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

def findDateOutliers(df, column_name, eps_setting = 0.05):
    datetime_data = df[column_name]

    light_outliers = DBSCANOutliers().find(datetime_data)
    
    return light_outliers

In [18]:
def fix_datetime_Column(df):
    data_cols = findColumns(df, ['date', 'Date', 'DATE'])
    
    for col in data_cols:
    
        df[col] = pd.to_datetime(df[col], errors='coerce')
        
        light_outliers = findDateOutliers(df, col)
        
        for item in light_outliers:
            
            df[col] = df[col].replace(item, None)
         
    return data_cols

In [19]:
from openclean.data.refdata import RefStore

refdata = RefStore()
city_df = refdata\
    .load('encyclopaedia_britannica:us_cities', auto_download=True)\
    .df()


In [20]:
# Cluster string using kNN clusterer (with the default n-gram setting)
# using the Levenshtein distance as the similarity measure.

from openclean.cluster.knn import knn_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan

def getClusters(df, col, minsize = 2, preds = 0.5): #ds
    dba = df.select(col).distinct()
    clusters = knn_clusters(
        values=dba,
        sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(preds)),
        minsize=minsize
    )
    return clusters

def print_cluster(cnumber, cluster):
    item_count = 0

def updateUsingClusters(df, col, clusters, isPrint = False): # ds,
    
    orignal_list = []
    suggestion_list = []
    clusters.sort(key=lambda c: len(c), reverse=True)
       
    for i, cluster in enumerate(clusters):        
        suggestion = cluster.suggestion()
        orignal_list = []
        suggestion_list = []
        if isPrint and i <5:
            print_cluster(i, cluster)
        
        for val, count in cluster.items(): 
            orignal_list.append(val)
            suggestion_list.append(suggestion)
    
    df[col] = df[col].replace(orignal_list, suggestion_list)

In [21]:
from openclean.function.eval.base import Col, Eval
from openclean.function.eval.logic import And
from openclean.function.value.phonetic import Soundex, soundex


def fix_city_and_name_Column(df, file): #ds,
    data_cols = findColumns(df, ['NAME', 'name', 'Name','city', 'City', 'CITY', 'BOROUGH', 'Borough', 'borough'])
    name_cols = findColumns(df, ['NAME', 'name', 'Name'])
    
    # mapping list to replace outliers
    outlier1 = ['', 'MR. ROSS ADAM C', 'MICHAEL', 'N. J.', 'WILLIAM 11', 'JOSEP;H``', 'DAID/11/2007', 'CHUNG   LUN', '718 9215010', 'ANTHONY', 'HSIA0-NAN', 'JOSEPH', '``````````', 'ROBERT  `', 'RAJENDRA9956700', '2', 'G.B.M.', 'EUGENE......JR', '6312100', 'CLAUDE,JR.', 'THOMAS``', 'ALAN  L', 'Nab53', 'MR. Y. B', 'J.J', 'PH8ILIP', 'I. M', 'RICHARD', 'ALBERTA S 111 D', 'P ;', 'GENECG.C. ENG &', 'J.J.', '2126202794', 'SHAW  HWA', 'HARRY         H', 'MR DOU8GLAS', '`1D', 'PAUL', 'K. T.', 'JOHN', '...NORMAN', 'EVAN   D', '7184361278BERNA', 'S.D. DON', 'KY00 SUK', 'JJ', 'YURI.`', 'MAD/Y/ARNI', 'ES ON SCH B', 'EUGENE.......JR', 'NEAL', 'F._ERIC', 'RYAN,  JR', 'AASDFASDFASDF', 'LA0-TECH', 'RODNEY   __', 'DAVID', 'G. L.', 'JAMES', 'LESLI8E', '7186054055', 'GEORGE', 'G.B.M', 'DAVID    JON', 'CHUNG---YAO', 'PETER', 'YUBUN(JACK)', 'GLEN A. L.', '1P', 'JUDE.....N.O', 'LEONARD--', 'WILLIAM', 'ANTHONY,111', 'WU(WOODY)', 'GAD/HON-AN', 'GLEN  A.L.', 'J.B. Jr.', 'LORENZO..A', 'J J', '..RAMSEY', 'HUI LI I', 'ANTONIO9', 'ROBERT', '0.BERT', 'DUMMY 2', '...JOSEPH', 'RUSSELL 111', 'THOMAS', 'H./E./CAMELLE', 'LALAL', 'M.E. P.E', 'R0OBIN VINCENT', '--young', 'AKM', 'LE1', 'IK.T.', 'LEO, JR.', 'J. Butch A. Jr.', 'WU (WOODY0', 'PAUL   N', 'CHRISTOPHER']
    mapping1 = [None, 'ROSS ADAM C', 'MICHAEL', 'N. J.', 'WILLIAM', 'JOSEPH', None, 'CHUNG LUN', None, 'ANTHONY', 'HSIA0 NAN', 'JOSEPH', None, 'ROBERT', 'RAJENDRA', None, 'G.B.M.', 'EUGENEJR', None, 'CLAUDE JR.', 'THOMAS', 'ALAN  L', 'Nab', 'MR. Y. B', 'J.J', 'PHILIP', 'I. M', 'RICHARD', 'ALBERTA', None, 'GENECG.C. ENG', 'J.J.', None, 'SHAW HWA', 'HARRYH', 'MR DOUGLAS', None, 'PAUL', 'K. T.', 'JOHN', 'NORMAN', 'EVAND', 'BERNA', 'S.D. DON', 'KY00 SUK', 'JJ', 'YURI.`', 'MADYARNI', 'ES ON SCH B', 'EUGENEJR', 'NEAL', 'FERIC', 'RYAN,  JR', 'AASDFASDFASDF', 'LA0 TECH', 'RODNEY', 'DAVID', 'G. L.', 'JAMES', 'LESLIE', None, 'GEORGE', 'G.B.M', 'DAVID JON', 'CHUNG YAO', 'PETER', 'YUBUN(JACK)', 'GLEN A. L.', None, 'JUDE N.O', 'LEONARD--', 'WILLIAM', 'ANTHONY,111', 'WU(WOODY)', 'GAD HON-AN', 'GLEN A.L.', 'J.B. Jr.', 'LORENZOA', 'J J', 'RAMSEY', 'HUI LI I', 'ANTONIO9', 'ROBERT', '0.BERT', 'DUMMY', 'JOSEPH', 'RUSSELL', 'THOMAS', 'H.E.CAMELLE', 'LALAL', 'M.E. P.E', 'R0OBIN VINCENT', 'young', 'AKM', 'LE1', 'IK.T.', 'LEO, JR.', 'J. Butch A. Jr.', 'WU (WOODY0', 'PAUL   N', 'CHRISTOPHER']

    outlier2 = ['SHARMA #0', "0'CONNOR", 'RUSHTON    UEL', 'UDDIN   Z', 'HINKLEY 1', 'O&#039;CONNOR, P.E.', '.OOK', 'SAMUELS111', 'O&#039;CONNOR', 'CALIENDO', 'SMITH   JR.', 'LO  BUE', '7AN', '+-+ETTIERI', 'SMITH, 111', 'KAMEN   1', '.EE', 'MASS, 1', '.EI', 'Zagaroli 3rd', 'RINI   II', 'KAMEN   R', 'RYAN 11', 'SPI8EZIA L S', 'MUFTIC..A.I.A', 'COSTELLO9 RA A I A', 'CALVANICO', 'LLC.', 'POEPPEL, P.E.', 'HAMA07', 'HINLEY,1', '1212', "O  ' CONNELL", 'HURT,JR.,', 'WESOLOWSKI', 'CHEN', '`ING, R.A', 'MARTARELLA 111', 'Gandhi, Ph.D., P.E.', '90I', 'ENNIS 2', 'COSTELLO R A A I A', '3UI', 'N/A', 'HURT,  JR', 'LEHR,1', 'KOHLER, 111', 'GERAZOUNIS', 'Alexander,1', 'LUBOW, R.A. LEED AP', 'RINI,111', '08CZAK', '````````````````````', 'CHAO  R.A.', 'Geier 11', '08NGEL', '08SOLOWSKI', 'I11', 'HINKLEY, 1', 'RUDIKOFF, P.E.', "O'CONNOR", 'SHAH   EZ', 'MIELE, JR., P.E.', 'RITTENHOUSE 111', 'AMADI   ISIOFIA', 'HINKLEY,1', 'RENFORE````````', "O'HARA,JR.", '73020012', 'PHAGOO   I', 'BRAY.....,', 'LLL', 'BHATHIA,1', 'GANDHI, PH. D., P.E', 'KO K', 'VASSALOTTI 11', 'HURT, JR .', '0018LKLE', 'RINI -111', 'PARIHAR', 'EE', 'L00802', 'ELISE.111', 'KING , R.A', 'CHRYSLER  P E', 'LEHR 1', 'Walters   Jr.', 'LEE', 'RINI  III', 'D&#039;ANGELO', '0UDOLPH III', 'VIEHE-NAESS 111', ',MO', '08E', '47DIKOFF', 'Yu,', '420865380', 'COPELAND', 'ZWIEFEL 3RD', 'PETERSEN', 'King, R.A.,', 'RINI, III', '7APA', 'CHEN   S', 'Hurt  Jr.', 'KATZ', 'NIZAMBAD.(P.E.)', '901BEN', '4153LOO', 'SYED-NAQVI', 'RYAN , JR.', 'K O K O R I S', 'ELISEO111', 'O&#039;CONNELL', 'ZEID61', '---Lewis', '00CHELI', 'MOHAMMAD       +++++', 'METZLER  P E', 'BAILEY', 'GANDHI, PH. D., P.E.', 'TIEMANN.111', 'SMITH.111', 'DI GER0NIMO', 'GANDHI, PH,D., P.E', 'III', 'J C', 'MAGAMI-QAIM-MAGAMI', '+M', 'LO G1UDICE', 'HOQUE', 'RUDIKOFF', 'Y10007OR', 'SMITH,111', 'KING R A FAIA', 'RYAN III, AIA', '08AN', 'STARK 1', 'MASS', 'VICTORI0, R.A', 'RIZVI   A', '21029677', "3'CONNOR", 'Wong /  Lai', 'KAPLAN 3', 'GRAICHEN.JR./DAWN/DI', 'GROSSMAN ,PE,F.A.C.I']
    mapping2 = ['SHARMA ', "CONNOR", 'RUSHTON UEL', 'UDDIN Z', 'HINKLEY ', 'CONNOR P.E.', None, 'SAMUELS', 'CONNOR', 'CALIENDO', 'SMITH JR.', 'LO BUE', None, 'ETTIERI', 'SMITH', 'KAMEN', '.EE', 'MASS', '.EI', 'Zagaroli', 'RINI', 'KAMEN R', 'RYAN', 'SPIEZIA L S', 'MUFTIC.A.I.A', 'COSTELLO9 RA A I A', 'CALVANICO', 'LLC.', 'POEPPEL P.E.', 'HAMA', 'HINLEY', None, "CONNELL", 'HURT JR.', 'WESOLOWSKI', 'CHEN', 'ING R.A', 'MARTARELLA', 'Gandhi', None, 'ENNIS ', 'COSTELLO R A A I A', None, None, 'HUR  JR', 'LEHR', 'KOHLER 111', 'GERAZOUNIS', 'Alexander', 'LUBOW R.A. LEED AP', 'RINI',None, None, 'CHAO R.A.', 'Geier', None, 'SOLOWSKI', None, 'HINKLEY', 'RUDIKOFF, P.E.', "CONNOR", 'SHAH EZ', 'MIELE JR. P.E.', 'RITTENHOUSE', 'AMADI   ISIOFIA', 'HINKLEY', 'RENFORE', "O'HARA,JR.", None, 'PHAGOO I', 'BRAY,', 'LLL', 'BHATHIA', 'GANDHI', 'KO K', 'VASSALOTTI', 'HURT JR.',None, 'RINI', 'PARIHAR', 'EE', None, 'ELISE', 'KING R.A', 'CHRYSLER  P E', 'LEHR', 'Walters Jr.', 'LEE', 'RINI  III', 'ANGELO', '0UDOLPH III', 'VIEHE-NAESS', 'MO', '08E', None, 'Yu,', None, 'COPELAND', 'ZWIEFEL 3RD', 'PETERSEN', 'King, R.A.,', 'RINI, III', '7APA', 'CHEN   S', 'Hurt  Jr.', 'KATZ', 'NIZAMBAD.(P.E.)', None, None, None, 'RYAN JR.', 'KOKORIS', 'ELISE', 'CONNELL', None, 'Lewis', 'CHELI', 'MOHAMMAD', 'METZLER  P E', 'BAILEY', 'GANDHI', 'TIEMANN', 'SMITH', 'DI GER0NIMO', 'GANDHI', 'III', 'J C', 'MAGAMI QAIM MAGAMI', None, 'LO G1UDICE', 'HOQUE', 'RUDIKOFF', None, 'SMITH', 'KING R A FAIA', 'RYAN III AIA', None, 'STARK', 'MASS', 'VICTORI0 R.A', 'RIZVIA', None, "CONNOR", 'Wong Lai', 'KAPLAN', 'GRAICHEN.JR. DAWN DI', 'GROSSMAN']

    outlier3 = ['', '....DEMO', '050069', 'DEM. CONTR.,', 'XXXXX', 'G/C 10114H9', 'CGWC10114H99', '00', 'X S000155', '082-36-1245', 'G.G', 'LESSEE', '......GC', "'", '..OWNER', 'GC 2293', '--', 'XXXXXX', 'LS 31,721', '...GC', 'gen.cont.', 'G.C TK#4592', 'PE', 'RLA - 818', '.....OWNER', 'RLA 16077', 'G C', 'X 4129892', 'G. C.', 'R.L.A', 'GC 1028350', 'WC10114H99', 'LEESEE', 'GEN.CONT.', 'SIGN..HANGER', 'DEMO 20451', 'D8615', '.X', 'P.L.L.C', '..DEMO', 'G .C', 'L A', 'G.C NY11101', '32820', '....OWNER', 'GC(DEMO)', 'C0NTRACTOR', 'EXPEDITORC99792', 'X 1341946', 'TRACK# 1390', 'EXPED.R4466', 'PLLC 9599691', 'G.C 1110101', '029649', '(CHECK)', 'DEM. CONTR,', 'EXPEDIT(H66172)', '.........GC', 'CITY OF N Y', 'GC 1170386', 'G. C', 'CO0OWNER', '(CHECKED)', 'C.C', '23392 1159774', 'DEMO {', 'RA', 'T. 31132', '....GC', 'RLA-787', 'TRACK #1390', 'D C', 'G.CONTR.', 'DEMO  CONT', '1GC', 'CC', 'demo G.C.', 'TRACK. #1390', 'M.F.S.P.C.', '...DEMO', 'DEMO G C', '13328', 'GEN  CONT', 'GC 1221073', "GC;'", 'DEMO 1341946', '11234', 'G.C.,', '.....GC', 'LIC.133668259 1', '?', '0WNER', 'C10892', 'GEN..CONT']
    mapping3 = [None, 'DEMO', None, 'DEM. CONTR', None, 'G/C', 'CGWC', None, 'X S', None, 'G.G', 'LESSEE', 'GC', None, 'OWNER', 'GC', None, None, 'LS ', 'GC', 'gen.cont.', 'G.C TK', 'PE', 'RLA ', 'OWNER', 'RLA ', 'G C', 'X', 'G. C.', 'R.L.A', 'GC', 'WC', 'LEESEE', 'GEN.CONT.', 'SIGN.HANGER', 'DEMO', None,None, 'P.L.L.C', 'DEMO', 'G.C', 'L A', 'G.C ', None, 'OWNER', 'GC(DEMO)', 'C0NTRACTOR', 'EXPEDITORC', None, 'TRACK', 'EXPED.R', 'PLLC ', 'G.C', None, None, 'DEM. CONTR,', 'EXPEDIT', 'GC', None, 'GC', 'G.C', 'CO0OWNER', None, 'C.C', None, 'DEMO', 'RA', None, 'GC', 'RLA', None, 'D C', 'G.CONTR.', 'DEMO  CONT', 'GC', 'CC', 'demo G.C.', None, 'M.F.S.P.C.', 'DEMO', 'DEMO G C', None, 'GEN  CONT', 'GC ', "GC ", 'DEMO ', None, 'G.C.', 'GC', 'LIC', None, '0WNER',None, 'GEN.CONT']

    outlier4 = ['', '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '99998', '000N/A', '65569+', '01827O', 'R9526', 'LP0256', 'N/A', '1964', 'ISLAND', '1609', '000PW1', '00DEMO', '0688.6', '00000', '.20929', 'LP0258', '000TOR', '0D8615', '0SWITA', '818', 'O02200', 'DEMO', '196', '1075', '0000NT', '215', '0', '00000`', "D'ALTO", '0455', '22377', 'DD8615', '050579', '226', 'SWITA', 'DD6815', 'X02689']
    mapping4 = [None, '0000GC', '083278', 'DD5615', '0000PB', '00ASB4', 'B81923', '099998', '000000', '065569', '01827O', '0R9526', 'LP0256',None, '001964',None, '001609', '000PW1', '00DEMO', '006886', '000000', '020929', 'LP0258', '000TOR', '0D8615', '0SWITA', '000818', 'O02200', None, '000196', '001075', '0000NT', '000215', '000000', '000000', None, '000455', '022377', 'DD8615', '050579', '000226', None, 'DD6815', 'X02689']

    outliers = outlier1+ outlier2+ outlier3+ outlier4
    mappings = mapping1+ mapping2+ mapping3+ mapping4
       
    for col in data_cols:
        
        df[col] = df[col].replace(outliers, mappings)
        
        '''
        if file != 'rbx6-tga4.tsv.gz':
            light_outliers = findDateOutliers(df, ds, col)

            for item in light_outliers:

                df[col] = df[col].replace(item, None)
        
        
        if file != 'hg8x-zxpr.tsv.gz' and col in name_cols:
            col_clusters = getClusters(df, ds, col)
            updateUsingClusters(df, ds, col, col_clusters, True)
         '''
        df[col] = df[col].astype('str')
        df[col] = df[col].str.upper()
        df[col] = df[col].replace(['N/A', 'NA','NONE', 'nan', 'NAN', 'NaN'], ["", "", "", "", "", ""])
        df[col] = df[col].str.replace(".", "", regex=False)
        df[col] = df[col].str.replace(",", "", regex=False)
        df[col] = df[col].str.replace(";", "", regex=False)
        df[col] = df[col].str.replace("!", "", regex=False)
        df[col] = df[col].str.replace("?", "", regex=False)
        df[col] = df[col].str.replace("_", "", regex=False)
        df[col] = df[col].str.replace("/", "", regex=False)
        df[col] = df[col].str.replace("\\", "", regex=False)
        
    return data_cols    

In [22]:
def saveDf(df, datafile):
    outputpath = datafile[2:11]+'_cleaned_data_improved.csv'
    df.to_csv(outputpath,sep=',',index=False,header=True) 

In [23]:
def dataCleanOnDataset(file):
    
    print('working on file: ', file)
    datafile, df = readData(file)
    
    cleaned_cols = []
    
    print("fixing Column Names.......")
    col_rename_dict = fixColumnNames(df)
    #print("Column renaming dictionary:")
    #print(col_rename_dict)
    
    df = df.rename(columns=col_rename_dict)

    print('fixing ID Number Columns......')
    cleaned_cols += fix_ID_Number_Column(df)
    
    print('fixing Binary Columns......')
    cleaned_cols += fix_Binary_Column(df)
    
    print('fixing Monetary Columns......')
    cleaned_cols += fix_Monetary_Column(df)
    
    print('fixing Numerical Columns......')
    cleaned_cols += fix_Numerical_Column(df)
    
    print('fixing Datetime Columns......')
    cleaned_cols += fix_datetime_Column(df)
    
    print('fixing City And Name Column......')
    cleaned_cols += fix_city_and_name_Column(df, file)
    
    saveDf(df, datafile)
    
    return cleaned_cols, datafile, df, col_rename_dict

In [24]:
def precision(tp, fp):
    return tp/(tp+fp)

def recall(tp, fn):
    return tp/(tp+fn)

In [25]:
def precision_recall(cleaned_columns, datafile, df, sample_size, col_rename_dict):
    df2  = pd.read_csv(datafile, dtype='object', sep='\t')
    df2 = df2.drop_duplicates()
    df2 = df2.rename(columns=col_rename_dict)
    

    df_sample_data =  df2.sample(sample_size).copy()
    df_new_sample_data = df.loc[df_sample_data.index].copy()
    print('sample size: ',sample_size)
    
    print('total size: ',sample_size * len(cleaned_columns))
    print('======================\n\n')

    same = 0

    for col in cleaned_columns:
        print("column: ", col)
        print("Original,\t Cleaned\n")
        for i in range(sample_size):
            df_sample_data.iloc[i].index
            if df_new_sample_data[col].iloc[i] == df_sample_data[col].iloc[i]:
                print(df_sample_data[col].iloc[i], '\t',df_new_sample_data[col].iloc[i], '\t')
                same += 1
            else:
                print(df_sample_data[col].iloc[i], '\t', df_new_sample_data[col].iloc[i], '\t*')

        print('*   ', same, ' same records   *\n')
        same = 0

        print('======================\n\n')

# DOB Permit Issuance

In [26]:
cleaned_cols, datafile, df, col_rename_dict = dataCleanOnDataset(file_list[0])
cleaned_cols = list(set(cleaned_cols))

print(cleaned_cols)

working on file:  ipu4-2q9a.tsv.gz
fixing Column Names.......
fixing ID Number Columns......
fixing Binary Columns......
fixing Monetary Columns......
fixing Numerical Columns......
fixing Datetime Columns......
fixing City And Name Column......
['Superintendent First & Last Name', "Permittee's Phone Number", 'Job Number', "Permittee's Last Name", "Owner's Business Name", "Site Safety Mgr's Last Name", 'Bin Number', "Site Safety Mgr's First Name", "Owner's Phone Number", "Owner's House City", "Owner's Last Name", 'BOROUGH', 'Issuance Date', 'Job Start Date', 'Expiration Date', "Permittee's Business Name", 'Site Safety Mgr Business Name', "Owner's House Street Name", "Owner's House Number", "Owner's First Name", "Permittee's First Name", 'Job doc Number', 'Permit Sequence Number', 'D O B Run Date', 'Nta Name', 'Filing Date', 'House Number', "Permittee's License Number", 'Superintendent Business Name', 'Street Name', 'Permit Si No']


In [27]:
precision_recall(cleaned_cols, datafile, df, 50, col_rename_dict)

sample size:  50
total size:  1550


column:  Superintendent First & Last Name
Original,	 Cleaned

RUSSELL ALEXANDER 	 RUSSELL ALEXANDER 	
STUART ISER 	 STUART ISER 	
nan 	  	*
PETER CERVINKA 	 PETER CERVINKA 	
BOGDAN KI 	 BOGDAN KI 	
HARVEY BROOKS 	 HARVEY BROOKS 	
nan 	  	*
nan 	  	*
JOYCE OCHAL 	 JOYCE OCHAL 	
nan 	  	*
nan 	  	*
JOHN WHITE 	 JOHN WHITE 	
GUARINO MASELLI 	 GUARINO MASELLI 	
nan 	  	*
REGINALD SAWYERS 	 REGINALD SAWYERS 	
nan 	  	*
nan 	  	*
HOWARD WALTERS 	 HOWARD WALTERS 	
CONSTANIN ZAHARIA 	 CONSTANIN ZAHARIA 	
nan 	  	*
LAWRENCE ZEUG 	 LAWRENCE ZEUG 	
nan 	  	*
YAU-SAN WU 	 YAU-SAN WU 	
nan 	  	*
nan 	  	*
nan 	  	*
JAWEOO JUHN 	 JAWEOO JUHN 	
nan 	  	*
ELY SAMUELS 	 ELY SAMUELS 	
nan 	  	*
nan 	  	*
nan 	  	*
SHIBBER KHAN 	 SHIBBER KHAN 	
nan 	  	*
nan 	  	*
EDGAR KRAMER 	 EDGAR KRAMER 	
nan 	  	*
nan 	  	*
nan 	  	*
nan 	  	*
nan 	  	*
FRANK COHEN 	 FRANK COHEN 	
nan 	  	*
LLOYD GOOD 	 LLOYD GOOD 	
nan 	  	*
nan 	  	*
nan 	  	*
JENNIFER HIXON 	 JENNIFER HIXON 	

In [46]:
tp = 0
fp = 0

fn = 0
tp +=  29 + 1 + 9 + 7 + 3 + 50 + 50 + 2 + 2 + 3 + 150 + 9 + 50 + 5 + 1 + 3 + 50 + 49 + 50 + 3 + 36

fp +=  1 + 1 + 3 + 1 + 1

fn +=  5 + 2 +1 +1

tn = 500 - tp - fp - fn

In [47]:
precision(tp, fp)

0.9876977152899824

In [48]:
recall(tp,fn)

0.9842381786339754

# DOB Cellular Antenna Filings

In [31]:
cleaned_cols, datafile, df, col_rename_dict = dataCleanOnDataset(file_list[1])
cleaned_cols = list(set(cleaned_cols))

print(cleaned_cols)

working on file:  iz2q-9x8d.tsv.gz
fixing Column Names.......
fixing ID Number Columns......
fixing Binary Columns......
fixing Monetary Columns......
fixing Numerical Columns......
fixing Datetime Columns......
fixing City And Name Column......
['Total Est Fee', 'City', 'Job Number', 'Applicant License Number', 'Initial Cost', 'First Permit  Date', 'Borough', "Owner's Business Name", 'Fee Status', 'Bin Number', "Owner's Last Name", "Owner's First Name", 'Latest Action Date', 'Doc Number', "Owner's  House Number", 'D O B Run Date', "Owner's  Phone Number", "Applicant's First Name", 'Pre- Filing Date', 'House Number', "Applicant's Last Name", 'Street Name']


In [32]:
precision_recall(cleaned_cols, datafile, df, 50, col_rename_dict)

sample size:  50
total size:  1100


column:  Total Est Fee
Original,	 Cleaned

389.5 	 389.5 	*
389.5 	 389.5 	*
338 	 338.0 	*
286.5 	 286.5 	*
255.6 	 255.6 	*
348.3 	 348.3 	*
441 	 441.0 	*
379.2 	 379.2 	*
513.1 	 513.1 	*
976.6 	 976.6 	*
441 	 441.0 	*
338 	 338.0 	*
245.3 	 245.3 	*
286.5 	 286.5 	*
595.5 	 595.5 	*
286.5 	 286.5 	*
513.1 	 513.1 	*
296.8 	 296.8 	*
215 	 215.0 	*
327.7 	 327.7 	*
389.5 	 389.5 	*
389.5 	 389.5 	*
441 	 441.0 	*
348.3 	 348.3 	*
389.5 	 389.5 	*
235 	 235.0 	*
389.5 	 389.5 	*
296.8 	 296.8 	*
255.6 	 255.6 	*
338 	 338.0 	*
338 	 338.0 	*
265.9 	 265.9 	*
410.1 	 410.1 	*
286.5 	 286.5 	*
389.5 	 389.5 	*
441 	 441.0 	*
338 	 338.0 	*
389.5 	 389.5 	*
1213.5 	 1213.5 	*
389.5 	 389.5 	*
286.5 	 286.5 	*
215 	 215.0 	*
441 	 441.0 	*
348.3 	 348.3 	*
338 	 338.0 	*
338 	 338.0 	*
389.5 	 389.5 	*
286.5 	 286.5 	*
1543.1 	 1543.1 	*
492.5 	 492.5 	*
*    0  same records   *



column:  City
Original,	 Cleaned

NEW YORK        	 NEW YORK        

In [49]:
tp = 0
fp = 0

fn = 0
tp += 50 + 50 + 50 + 27 + 50 + 50 + 50

fp += 2 + 50

fn += 3 +2

In [50]:
precision(tp, fp)

0.862796833773087

In [51]:
recall(tp,fn)

0.9849397590361446

# Open Restaurant Applications

In [36]:
cleaned_cols, datafile, df, col_rename_dict = dataCleanOnDataset(file_list[2])
cleaned_cols = list(set(cleaned_cols))

print(cleaned_cols)

working on file:  pitm-atqc.tsv.gz
fixing Column Names.......
fixing ID Number Columns......
fixing Binary Columns......
fixing Monetary Columns......
fixing Numerical Columns......
fixing Datetime Columns......
fixing City And Name Column......
['Restaurant Name', 'Sidewalk Dimensions (Length)', 'Legal Business Name', 'Building Number', 'Borough', 'Food Service Establishment Permit Number', 'SLA Serial Number', 'Roadway Dimensions (Length)']


In [37]:
precision_recall(cleaned_cols, datafile, df, 50, col_rename_dict)

sample size:  50
total size:  400


column:  Restaurant Name
Original,	 Cleaned

Cachapa's Brooklyn 	 CACHAPA'S BROOKLYN 	*
CRYSTAL'S RESTAURANT & BISTRO INC 	 CRYSTAL'S RESTAURANT & BISTRO INC 	
so pho 88 inc 	 SO PHO 88 INC 	*
BOHEMIAN SPIRIT RESTAURANT 	 BOHEMIAN SPIRIT RESTAURANT 	
MOMENTEA KITCHEN 	 MOMENTEA KITCHEN 	
Hotel Chantelle 	 HOTEL CHANTELLE 	*
Town Stages 	 TOWN STAGES 	*
ssam korean restaurant 	 SSAM KOREAN RESTAURANT 	*
Guyers 	 GUYERS 	*
Pollos a la Brasa Mario 	 POLLOS A LA BRASA MARIO 	*
Black Star Bakery&Cafe 	 BLACK STAR BAKERY&CAFE 	*
Queen's Room 	 QUEEN'S ROOM 	*
Thyme and Tonic 	 THYME AND TONIC 	*
Cagen 	 CAGEN 	*
LA DULCE BAKERY 	 LA DULCE BAKERY 	
108 Food Dried Hot Pot 	 108 FOOD DRIED HOT POT 	*
El Pollo Inka Peru 	 EL POLLO INKA PERU 	*
The wilky 	 THE WILKY 	*
Antiochia Grill 	 ANTIOCHIA GRILL 	*
RAICES COLOMBIANAS 	 RAICES COLOMBIANAS 	
The Commissioner Tavern 	 THE COMMISSIONER TAVERN 	*
Joe and Sal's Pizzeria 	 JOE AND SAL'S PIZZERIA 	*
Sichuan hotp

In [52]:
tp = 0
fp = 0
fn = 0
tp += 38 + 50 + 37 + 8 + 31+ 50

fp += 1

fn += 1 + 2

In [53]:
precision(tp, fp)

0.9953488372093023

In [54]:
recall(tp,fn)

0.9861751152073732

# DOB NOW: Safety – Facades Compliance Filings

In [41]:
cleaned_cols, datafile, df, col_rename_dict = dataCleanOnDataset(file_list[3])
cleaned_cols = list(set(cleaned_cols))

print(cleaned_cols)

working on file:  xubg-57si.tsv.gz
fixing Column Names.......
fixing ID Number Columns......
fixing Binary Columns......
fixing Monetary Columns......
fixing Numerical Columns......
fixing Datetime Columns......
fixing City And Name Column......
['Qewi Bus Name', 'Qewi Bus Street Name', 'Owner Bus Street Name', 'Owner City', 'Qewi Name', 'Sequence No', 'Field Inspection Completed Date', 'Qewi Nys Lic No', 'BOROUGH', 'Qewi Signed Date', 'House No', 'Qewi City', 'Prior Cycle Filing Date', 'Owner Bus Name', 'Filing Date', 'Owner Name', 'Control No', 'Tr6 No', 'Street Name']


In [42]:
precision_recall(cleaned_cols, datafile, df, 50, col_rename_dict)

sample size:  50
total size:  950


column:  Qewi Bus Name
Original,	 Cleaned

RONNETTE RILEY ARCHITECT 	 RONNETTE RILEY ARCHITECT 	
VIDARIS, INC 	 VIDARIS INC 	*
HOFFMAN ARCHITECTS, INC 	 HOFFMAN ARCHITECTS INC 	*
nan 	  	*
DARIUS TORABY ARCHITECTS P.C 	 DARIUS TORABY ARCHITECTS PC 	*
OLD STRUCTURES ENGINEERING 	 OLD STRUCTURES ENGINEERING 	
nan 	  	*
PM ARCHITECTS, LLP 	 PM ARCHITECTS LLP 	*
WALTER B. MELVIN ARCHITECTS, LLC 	 WALTER B MELVIN ARCHITECTS LLC 	*
nan 	  	*
JMA CONSULTANTS & ENGINEERS PC 	 JMA CONSULTANTS & ENGINEERS PC 	
nan 	  	*
JOHN J FLYNN CONSULTING ENGR 	 JOHN J FLYNN CONSULTING ENGR 	
JMA CONSUTLTANTS&ENGINEERS PC 	 JMA CONSUTLTANTS&ENGINEERS PC 	
MJ MACALUSO & ASSOCIATES 	 MJ MACALUSO & ASSOCIATES 	
LINGINEERING CONSULTING, LLC 	 LINGINEERING CONSULTING LLC 	*
SUPERSTRUCTURES ENG. & ARCH 	 SUPERSTRUCTURES ENG & ARCH 	*
RAND ENGINEERING & ARCHITECT,PC 	 RAND ENGINEERING & ARCHITECTPC 	*
nan 	  	*
HAKS ENGINEERS, ARCHITECTS AND L 	 HAKS ENGINEERS ARCHITECTS AND L 	

In [56]:
tp = 0
fp = 0

fn = 0
tp += 31 + 13 + 50 + 50 + 9 + 36 + 44 + 25 + 11 + 34 + 28 + 39 + 44

fp += + 9 + 14 + 15 + 16 + 11

fn += 1 + 3 + 2 + 1 + 2


In [57]:
precision(tp, fp)

0.8643006263048016

In [58]:
recall(tp,fn)

0.9787234042553191